# QuDiPy tutorial: Control pulses

This tutorial will walk you through how to create and use a control pulse object. Some of the information presented here also appears in the Loading_.ctrlp_and_.qcirc_files tutorial.


## 1. First load the relevant modules

In [ ]:
# Since we don't actually have a package yet for people to install.. 
# We need to add our folder to the PYTHONPATH in order for import to find qudipy
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import qudipy.circuit as circ
import numpy as np

## 2. Loading control pulse files
First, we will demonstrate how to load a control pulse from a file. Control pulse files are denoted by the .ctrlp file extension. There are some dummy control pulse files located in the tutorial data directory. We will use the `CTRLZ_3_4.ctrlp` file.

In [ ]:
# Get the directory where .ctrlp files are loaded
pulse_dir = os.getcwd()+"/QuDiPy tutorial data/Control pulses/"

pulse_file = pulse_dir + "CTRLZ_3_4.ctrlp"

Before we actually load the .ctrlp files, let's see what the contents of an example .ctrlp file look like.

In [ ]:
with open(pulse_file, 'r') as f:
    print(f.read())

Here we are looking at the `CTRLZ_3_4.ctrlp` file. There are three lines at the start of the file to describe the control pulse.  
- The first line, "Ideal gate:" specifies what is the equivalent ideal gate being implemetned by the control pulse.  This is used to simulate the ideal circuit later when dealing with QuantumCircuit objects (see Quantum_circuit tutorial).
- The second line, "Pulse type:" sepcifies whether the control pulse is described using experimental parameters (such as gate voltages and magnetic fields) or effective parameters (such as g, J, and things directly implemented in the effective Hamiltonian).
- The third line, "Pulse length:" specifies the length of the pulse in ps.

After those are specified, "Control pulses:" indicates the actual control pulse is going to be given. The next line contains the name of each control variables (V1, V2, ...). The next lines specify the acutal values for the control pulse. The control pulse will be linearly interpolated during the actual time simulation using the control pulse.  The first three lines of the .ctrlp file can appear in any order as long as "Control pulses:" is the last portion of the pulse file. 

Now, let's go ahead and load this pulse. This is achieved using the `load_pulses` method which returns a dictionary of composed of each `ControlPulse` object loaded. `load_pulses` can handle multiple .ctrlp inputs at once.

In [ ]:
pulse_dict = circ.load_pulses(pulse_file) 

Let's check the returned dictionary from `load_pulses` and see what a `ControlPulse` object looks like.

In [ ]:
pulse_dict

As you can see, load_pulses returns a dictionary of `ControlPulse` objects. Each `ControlPulse` contains the same information that was specified in the .ctrlp file.

In [ ]:
print("Name:",pulse_dict['CTRLZ_3_4'].name)
print("Pulse type:",pulse_dict['CTRLZ_3_4'].pulse_type)
print("Length:",pulse_dict['CTRLZ_3_4'].length)
print("Ideal gate:",pulse_dict['CTRLZ_3_4'].ideal_gate)
print("Number of control variables:",pulse_dict['CTRLZ_3_4'].n_ctrls)
print("Control variable names:",pulse_dict['CTRLZ_3_4'].ctrl_names)
print("Control pulse values:",pulse_dict['CTRLZ_3_4'].ctrl_pulses)

This control pulse is relatively boring, so let's make a custom one to show other features of the `ControlPulse` class as well as a different way you can define a `ControlPulse` object.

## 3. Building a custom control pulse object
The first step in making your own control pulse object is to initialize it.  We will initialize a control pulse with the parameters: 
- name = "control_tut"
- type = "effective"
- pulse lengt = 5 ns

In [ ]:
ctrl_p = circ.ControlPulse(pulse_name="Pulse tutorial", pulse_type="effective", pulse_length=5e-9)

Of course, so far there is nothing interesting with this pulse as it has no actual pulse associated with it. In order to make it interesting, we need to add control variables to the pulse. Each added variable will have it's own individual pulse sequence which when taken together, constitutes the entire control pulse.

In [ ]:
# Let's define our first control variable, V1
# This will just be a linear sweep from 0 -> 1
V1 =  np.linspace(0,1,100)
ctrl_p.add_control_variable(var_name="V_1", var_pulse=V1)

# Again, let's add another control variable, V2
# This one will actual remain constant throughout the pulse
V2 = np.linspace(0.5,0.5,100)
ctrl_p.add_control_variable(var_name="V_2", var_pulse=V2)

We can visualize the created control pulse using the `plot()` method of the class. We will go over the plot method more towards the end of this tutorial.

In [ ]:
ctrl_p.plot()

Let's add a third control variable. However, notice how when we created V1 and V2, both variables had the same number of pulse points (in this case 100). The `ControlPulse` object requires that any added control variables have the same number of points as the already added control variables. See what happens when we try to add a third variable with a different number of points.

In [ ]:
V3 = np.linspace(2,0,8) # Note we are trying to add a variable with 105 pulse points
ctrl_p.add_control_variable(var_name="V_3", var_pulse=V3)

Of course, if we only use 5 points, then it works just fine.

In [ ]:
V3 = np.linspace(2,0,100)
ctrl_p.add_control_variable(var_name="V_3", var_pulse=V3)
ctrl_p.plot()

Because we only specified the pulse length when we initialized the object, the control variable pulse points are defaulted to be linearly spaced in time. We can specify the time axis though using the 'time' variable when adding a control variable.

In [ ]:
t = np.concatenate([np.array([0]),np.geomspace(10E-12,5E-9,99)])
ctrl_p.add_control_variable(var_name="time", var_pulse=t, overwrite=False)
ctrl_p.plot()

Notice now how the specified points are not linearly spaced in time, but rather are spaced according to the spacing we specified in the `t` variable.

If you want to obtain the interpolated control variables at arbitrary time values, this is achieved simply by calling the control pulse object. Either a single time points can be inputted or a list of multiple time points can be supplied. All time points must be within the time interval specified by the `time` control variable.

In [ ]:
ctrl_p(2E-9)

In [ ]:
ctrl_p([2E-9,2.5E-9,3E-9])

## 4. Define inner control variable mapping
Typically, experiments are done using a set of accessible experimental control variables. For a quantum dot device, such variables are the gate voltages applied to the gate electrodes which define the quantum dots. These voltages are used to tune the potential landscape of the device and subsequently control the dynamics of the electrons trapped within these quantum dots.

Of course, when you actually look at an effective spin Hamiltonian for a series of electrons trapped within a quantum dot network, the relevant Hamiltonian parameters are the spin-exchange terms (typically denoted by J) or ESR magnetic field terms. These are what are used in the spin simulator of this QuDiPy module. If you want to simulate an experiment on a quantum dot device, which control variables are better to use? Gate voltages or the effective Hamiltonian parameters?

There is no correct answer. However, it is certainly reasonable to see the benefit in being able to specify control pulses directly in terms of gate voltages and subsequently map them to effective Hamiltonian parameters for use in an effective spin Hamiltonian simulation. This section outlines how we can do such a thing, by defining a mapping between an outer layer of experimentally accessible control variables (i.e. gate voltages) and an inner layer of effective Hamiltonian variabels (i.e. exchange interactions).

This is easily achieved using the `define_ctrl_mapping` method.

In [ ]:
# First, we will define some grid vectors for the variables that currently compose our control pulse
V1_grid = np.linspace(0,2,50)
V2_grid = np.linspace(0,2,50)
V3_grid = np.linspace(0,2,50)

# Now, we will specify a function f(V1,V2,V3) which maps the outer control variables to a 
# inner control variable.
# Note that this is a dummy function for this tutorial.
inner_var1 = V1_grid[:,None,None]**2 + 3*V2_grid[None,:,None]**2 + \
    0.5*V3_grid[None,None,:]**2

# Now we will define the control mapping to the first inner variable which we label 'J_1'
ctrl_p.define_ctrl_mapping((V1_grid,V2_grid,V3_grid), inner_var1, 'J_1')

ctrl_p.plot(show_inner=True)

Now the control pulse has two layers of variables associated with it. The outer one is in terms of V_i's as shown above while the other one is in terms of an inner layer of variables (currently only J_1)

We can add additional inner control variables by defining additional control mappings for them.

In [ ]:
inner_var2 = 3*V1_grid[:,None,None]**2 + 0.5*V2_grid[None,:,None]**2 + \
    V3_grid[None,None,:]**2
ctrl_p.define_ctrl_mapping((V1_grid,V2_grid,V3_grid), inner_var2, 'J_2')

ctrl_p.plot(show_inner=True)

Now when we call the interpolated control pulse, we can specify whether or not we want to return the inner or outer control variables. If a control mapping is defined, then default behavior is to return the inner control variables (i.e. J_i's). See the following example.

In [ ]:
# Since a control mapping is defined, the default behavior will return the inner control variables
# Notice that only two control variables are returned corresponding to J_1 and J_2
ctrl_p([2E-9,2.5E-9,3E-9,3.8E-9])

In [ ]:
# Now we will specify that we do not want to return the inner control variables.
# Notice that now three control variables are returned corresponding to V_1, V_2 and V_3
ctrl_p([2E-9,2.5E-9,3E-9,3.8E-9], call_inner=False)

## 5. Plotting control pulses

So far, we have only done basic plotting with the control pulse object, but there are a few other keyword arguments we will go over before ending the tutorial. First, look at the docstring of the `plot` method. Then, we will show several examples highlighting the different keyword arguments.

In [ ]:
print(ctrl_p.plot.__doc__)

In [ ]:
# plot_ctrls can specify individual control variables to plot
ctrl_p.plot(plot_ctrls=['V_1','V_3'])

In [ ]:
# time_int specifies the interval over which to plot the control pulse
# n specifies the number of time points to plot in the interval.
# Both of these must be used together. If n is not specified, then time_int will do nothing.
ctrl_p.plot(time_int=[1E-9,2.3E-9], n=20)

In [ ]:
# As seen earlier, show_inner will show the inner control variables
ctrl_p.plot(show_inner=True)

In [ ]:
# show_base will just plot the individual time points of the pulse and not a line.
# This is useful when debugging the individual time points of the control pulse variables.
ctrl_p.plot(show_base=True)